In [2]:
import time
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bt
import seaborn as sns
sns.set(style="darkgrid")

atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Release"
sys.path.append(atlas_path)

from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *

In [3]:
tickers_new = 'aapl,msft,c,gs,ge,jnj,pg,ko,amzn,jpm,adbe,ma,dis,txn'
data = bt.get(
    tickers_new,
    start='2010-01-01',
    end = '2024-01-01'
)
data.tail()

out_path = r'C:\Users\natha\OneDrive\Desktop\Rust\atlas\crates\atlas\src\test\data\sp500'
for col in data.columns:
    df_ticker = pd.DataFrame(data[col])
    df_ticker.columns = ["close"]
    df_ticker.to_csv(os.path.join(out_path, col + '.csv'))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
exchange_path = r'C:\Users\natha\OneDrive\Desktop\Rust\atlas\crates\atlas\src\test\data\sp500'
strategy_id = "test_strategy"
exchange_id = "test_exchange"
portfolio_id = "test_portfolio"

hydra = Hydra()
intial_cash = 100.0
exchange = hydra.addExchange(exchange_id, exchange_path, "%Y-%m-%d")
portfolio = hydra.addPortfolio(portfolio_id, exchange, intial_cash)
hydra.build()

In [10]:
close = AssetReadNode.make("close", 0, exchange)
ma = MeanObserverNode(close, 50)
spread = AssetOpNode.make(close, ma, AssetOpType.SUBTRACT)
filter = ExchangeViewFilter(ExchangeViewFilterType.GREATER_THAN, 0.0, None)
ev = ExchangeViewNode.make(exchange, spread, filter)

allocation = AllocationNode.make(
    ev
)
strategy_node_signal = StrategyNode.make(allocation, portfolio)
strategy = hydra.addStrategy(Strategy(strategy_id, strategy_node_signal, 1.0), True)
strategy.enableTracerHistory(TracerType.NLV)

In [12]:
st = time.perf_counter_ns()
hydra.run()
et = time.perf_counter_ns()
print(f"Time to Run: {(et-st)/1e6}ms")

Time to Run: 0.8227ms


In [14]:
nlv = strategy.getHistory(TracerType.NLV)
returns = (nlv[-1] - nlv[0] ) / nlv[0]
print(f"Returns: {returns}")

Returns: 4.427639459014664
